In [74]:
import pingouin as pg
import pandas as pd
import numpy as np

In [75]:
data = pd.read_csv("task-2-data.csv", sep=",")
data["minutes"] = pd.to_numeric(data["minutes"])
data

,factor1,factor2,minutes
0,rano,ticho,6
1,rano,ticho,8
2,rano,ticho,11
3,rano,hudba,7
4,rano,hudba,8
5,rano,hudba,12
6,rano,hudba,10
7,rano,hluk,8
8,rano,hluk,7
9,rano,hluk,20


In [76]:
value_counts = data.groupby(["factor1", "factor2"]).size().unstack()
total_number_cols = value_counts.sum(axis=0)
total_number_rows = value_counts.sum(axis=1)
N = value_counts.values.sum()
value_counts


factor2,hluk,hudba,krik,ticho
factor1,,,,
poledne,4,3,2,3
rano,3,4,3,3
vecer,3,4,5,3


In [77]:
value_sum = data.groupby(["factor1", "factor2"]).sum().unstack()

Cx = np.square(value_sum.values.sum()) / N
print(f"Total number of numbers: {N}")
print(f"Correction term Cx: {Cx}")
value_sum

Total number of numbers: 40
Correction term Cx: 5267.025


minutes                 
factor2    hluk hudba krik ticho
factor1                         
poledne      51    23   25    28
rano         35    37   45    25
vecer        39    45   85    21

In [78]:
SSt = np.square(data["minutes"].values).sum() - Cx
SSt

781.9750000000004

In [79]:
SSc = 0
for col in value_counts.columns.to_list():
    SSc += np.square(value_sum["minutes"][col].sum()) / value_counts[col].sum()
SSc -= Cx
SSc

308.69217171717173

In [80]:
SSr = 0
for row in value_counts.index.to_list():
    SSr += np.square(value_sum.loc[row].sum()) / value_counts.loc[row].sum()
SSr -= Cx
SSr

34.80192307692323

In [81]:
g_square = np.square(value_sum)
g_square.columns = g_square.columns.droplevel()
SSg = g_square.div(value_counts).values.sum() - Cx - SSr - SSc
SSg

29.06423853923843

In [82]:
SSe = SSt - SSc - SSr - SSg
SSe

409.41666666666697

In [88]:
index = ["Rows", "Columns", "Within Groups", "Error", "Total"]
columns = ["Sum of Squares", "Degrees of Freedom", 
           "Mean of Squares", "F", "P-value"]
df = pd.DataFrame(index=index,columns=columns)
df["Sum of Squares"] = [SSr, SSc, SSg, SSe, SSt]
R = len(value_counts.index.to_list())
C = len(value_counts.columns.to_list())
MSr, MSc, MSg, MSe = SSr/(R-1), SSc/(C-1), SSg/((C-1)*(R-1)), SSe/(N-C*R)
Fr, Fc, Fg = MSr/MSe, MSc/MSe, MSg/MSe
df["Degrees of Freedom"] = [R-1, C-1, (R-1)*(C-1), N-C*R, N-1]
df["Mean of Squares"] = [MSr, MSc, MSg, MSe, np.NAN]
df["F"] = [Fr, Fc, Fg, np.NAN, np.NAN]
df["P-value"] = [0.31912707, 0.00113581, 0.91470098, np.NAN, np.NAN]
df


,Sum of Squares,Degrees of Freedom,Mean of Squares,F,P-value
Rows,34.801923,2,17.400962,1.190052,NaN
Columns,308.692172,3,102.897391,7.037151,NaN
Within Groups,29.064239,6,4.844040,0.331284,NaN
Error,409.416667,28,14.622024,NaN,NaN
Total,781.975000,39,NaN,NaN,NaN


In [ ]:
result = data.anova(ss_type=2, dv="minutes", between=["factor1", "factor2"], detailed=True).round(3)
alpha = 0.05
result

,Source,SS,DF,MS,F,p-unc,np2
0,factor1,16.562,2.0,8.281,0.566,0.574,0.039
1,factor2,290.452,3.0,96.817,6.621,0.002,0.415
2,factor1 * factor2,47.305,6.0,7.884,0.539,0.774,0.104
3,Residual,409.417,28.0,14.622,NaN,NaN,NaN
